In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import numpy as np;
import os
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
#数据预处理函数，在dir文件夹下每个子文件是一类内容

def datahelper(dir):
#返回为文本，文本对应标签，标签及索引，索引及标签
    
    labels_index={}
    index_lables={}
    fs = os.listdir(dir)

    i = 0;
    for f in fs:
        labels_index[f[:-4]] = i;
        index_lables[i] = f[:-4]
        i = i + 1;
    print(labels_index)
    
#     texts = []   # 每句话（jieba拆词后）的列表
    labels = []  # list of label ids
    X = []
    
    for la in labels_index.keys():
        print(la + " " + str(labels_index[la]))
        la_dir = 'D:/fonts' + "/" + la+'.csv'
        font_datas = np.array(pd.read_csv(la_dir).iloc[:,12:])
        for cnt in range(len(font_datas)):
            labels.append(labels_index[la])
            X.append(font_datas[cnt])
            
#         for f in fs:
#             file = open(la_dir + "/" + f, encoding='utf-8') # 打开该txt文件
#             lines = file.readlines();
#             text = ''
#             num_recs=0
#             for line in lines:
#                 if len(line) > 5: # 只提取大于5个字的语句
#                     line = extract_chinese(line) # 提取中文有效信息，汉字 字母 数字
#                     words = jieba.lcut(line, cut_all=False, HMM=True) # 对每句话拆分成词语的list
#                     text = words
#                     texts.append(text)
#                     labels.append(labels_index[la])
#                     num_recs = num_recs + 1
    return np.array(X),labels,labels_index,index_lables

train_dir = 'D:/fonts'
# train_dir = 'D:/THUCNews'
# train_dir = 'D:/THUCNewsSmall'

X,labels,labels_index,index_lables = datahelper(train_dir)

{'AGENCY': 0, 'ARIAL': 1, 'BAITI': 2, 'BANKGOTHIC': 3, 'BASKERVILLE': 4, 'BAUHAUS': 5, 'BELL': 6, 'BERLIN': 7, 'BERNARD': 8, 'BITSTREAMVERA': 9, 'BLACKADDER': 10, 'BODONI': 11, 'BOOK': 12, 'BOOKMAN': 13, 'BRADLEY': 14, 'BRITANNIC': 15, 'BROADWAY': 16, 'BRUSH': 17, 'BUXTON': 18, 'CAARD': 19, 'CALIBRI': 20, 'CALIFORNIAN': 21, 'CALISTO': 22, 'CAMBRIA': 23, 'CANDARA': 24, 'CASTELLAR': 25, 'CENTAUR': 26, 'CENTURY': 27, 'CHILLER': 28, 'CITYBLUEPRINT': 29, 'COMIC': 30, 'COMMERCIALSCRIPT': 31, 'COMPLEX': 32, 'CONSOLAS': 33, 'CONSTANTIA': 34, 'COOPER': 35, 'COPPERPLATE': 36, 'CORBEL': 37, 'COUNTRYBLUEPRINT': 38, 'COURIER': 39, 'CREDITCARD': 40, 'CURLZ': 41, 'DUTCH801': 42, 'E13B': 43, 'EBRIMA': 44, 'EDWARDIAN': 45, 'ELEPHANT': 46, 'ENGLISH': 47, 'ENGRAVERS': 48, 'ERAS': 49, 'EUROROMAN': 50, 'FELIX TITLING': 51, 'FOOTLIGHT': 52, 'FORTE': 53, 'FRANKLIN': 54, 'FREESTYLE': 55, 'FRENCH': 56, 'GABRIOLA': 57, 'GADUGI': 58, 'GARAMOND': 59, 'GEORGIA': 60, 'GIGI': 61, 'GILL': 62, 'GLOUCESTER': 63, 'GOTHI

In [3]:
# textCNN模型构造 - BN处理
class fontCNN(nn.Module):
    def __init__(self):
        super(fontCNN, self).__init__()
        #需要将事先训练好的词向量载入
        self.conv1 = nn.Sequential( # 1*20*20
                      nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5,stride=1, padding=2),
                      nn.BatchNorm2d(num_features=16, eps=1e-05, momentum=0.1, affine=True), # BN 处理
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2) # (16,10,10)
                     )
        self.conv2 = nn.Sequential(
                      nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
                      nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1, affine=True), # BN 处理
                      nn.ReLU(),
                      nn.MaxPool2d(2) # 32 5 5
                     )
        self.conv3 = nn.Sequential(
                      nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2),
                      nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1, affine=True), # BN 处理
                      nn.ReLU(),
                      nn.MaxPool2d(2) # 64 2 2
        )
        self.out = nn.Linear(64*2*2, len(set(labels)))

    def forward(self, x):
#         x=x.view(x.size(0),1,max_len,word_dim)
        #print(x.size())
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1) # 将（batch，outchanel,w,h）展平为（batch，outchanel*w*h）
        #print(x.size())
        output = self.out(x)
        return output

In [5]:
# 读取空模型
# cnn = fontCNN()

# 加载原有模型
# 有BN层
cnn = fontCNN()
cnn.load_state_dict(torch.load('C:/Users/jxjsj/Desktop/JupyterHome/DLmodel/fontCNN_BN.pkl'))

In [6]:
#构建textCNN模型超参数与数据封装入加载器 - LSC

#损失函数
loss_function = nn.CrossEntropyLoss()

#划分训练数据和测试数据
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

x_train = torch.Tensor(x_train)
y_train = torch.LongTensor(y_train)
x_test = torch.Tensor(x_test)
y_test = torch.LongTensor(y_test)

train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=5000,shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2000,shuffle=True)

In [9]:
# 训练 - LSC

use_gpu = True

LR = 0.00007
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)

if use_gpu:
    cnn = cnn.cuda()
else:
    cnn = cnn.cpu()

for epoch in range(10):
    print('epoch {}'.format(epoch + 1))
    # training-----------------------------
    cnn.train()
    train_acc = 0.

    for step, (batch_x, batch_y) in enumerate(train_data_loader):
        batch_x, batch_y = Variable(batch_x), Variable(batch_y)
        
        batch_x = batch_x.view(batch_x.size(0),1,20,20)
        
        if use_gpu:
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()
            
        out = cnn(batch_x)
        loss = loss_function(out, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pred = torch.max(out, 1)[1]
        num_correct = (pred == batch_y).sum()
        train_acc += num_correct.data
        
#         print('Step:',step+1,'Finished!')
    print('Train Acc: {:.6f}'.format(train_acc.cpu().numpy() / (len(train_dataset))))
#     print(classification_report(L_train_real,L_train_pred))

    # evaluation--------------------------------
    cnn.eval()
    with torch.no_grad():
        eval_acc = 0.

        for batch_x, batch_y in test_data_loader:
            batch_x, batch_y = Variable(batch_x), Variable(batch_y)
            
            batch_x = batch_x.view(batch_x.size(0),1,20,20)

            if use_gpu:
                batch_x = batch_x.cuda()
                batch_y = batch_y.cuda()

            out = cnn(batch_x)
            loss = loss_function(out, batch_y)
            
            pred = torch.max(out, 1)[1]
            num_correct = (pred == batch_y).sum()
            eval_acc += num_correct
            
        print('Test Acc: {:.6f}'.format(eval_acc.cpu().numpy() / (len(test_dataset))))
#         print(classification_report(L_val_real,L_val_pred))

epoch 1
Train Acc: 0.614202
Test Acc: 0.579683
epoch 2
Train Acc: 0.614561
Test Acc: 0.579269
epoch 3
Train Acc: 0.614667
Test Acc: 0.580134
epoch 4
Train Acc: 0.614750
Test Acc: 0.579449
epoch 5
Train Acc: 0.614924
Test Acc: 0.579804
epoch 6
Train Acc: 0.614874
Test Acc: 0.579954
epoch 7
Train Acc: 0.615365
Test Acc: 0.580098
epoch 8
Train Acc: 0.615511
Test Acc: 0.579575
epoch 9
Train Acc: 0.615645
Test Acc: 0.579713
epoch 10
Train Acc: 0.615548
Test Acc: 0.580344


In [15]:
# 有BN层
torch.save(cnn.state_dict(),'C:/Users/jxjsj/Desktop/JupyterHome/DLmodel/fontCNN_BN.pkl')